In [ ]:
%matplotlib inline

#
# import utilitaires
#
import cv2
import h5py
import numpy as np
import pandas as pd
import random
import seaborn as sns

from dataset_helpers.MetaObject import MetaObject
from dataset_helpers.PlantVillage import PlantVillageConfig, plant_village_load
from jupyter_helpers import display_html
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

# faire en sorte que random soit deterministe
random.seed(42)

In [ ]:
#
# configurer les divers processus
#
pv_config = PlantVillageConfig()

config_overrides = MetaObject.from_json("config_overrides.json")
if not config_overrides is None:
    MetaObject.override_from_object(pv_config, 
                                    config_overrides.dataset)

In [ ]:
#
# obtenir le dataset PlantVillage
#
pv_dataset = plant_village_load(pv_config)
if pv_dataset is None:
    display_html(f"<b>Invalid dataset</b>")
else:
    pv_h5 = pv_dataset.h5_file
    pv_dataframe = pd.DataFrame(pv_dataset.data)

In [ ]:
i = 0

def do_nothing(o):
    global i
    i += 1

# lent; glob va ~2x plus vite
# visititems TRES lent; glob va un ordre de grandeur plus vite
pv_h5["/images"].visit(do_nothing)

print(i)

In [ ]:
i = 0
for file in glob(r'C:\Users\jfgag\Projects\PlantVillage\dataset\Plant_leave_diseases_dataset_with_augmentation\**\*.*'):
    i += 1
print(i)

In [ ]:
i = 0
with tqdm(total=61525) as progress:
    def do_nothing(o):
        global i
        i += 1
        # ce qui est tres lent et de mettre a jour les string de display
        # juste update est ben correcte
        progress.update()

    pv_h5["/images"].visit(do_nothing)

print(i)

# Exploration

In [ ]:
display_html(f"<b>{pv_config.install_path}</b> - data {pv_dataset.shape}")
display(pv_dataset.head(5))

In [ ]:
display_html(f"<b>{pv_config.install_path}</b> - info")
pv_dataset.info()

In [ ]:
species = pv_dataset["species"].unique()
species.sort()
display_html(f"<b>Modalites de 'species'</b> - count: {species.size}")
display(list(species))

In [ ]:
labels = pv_dataset["label"].unique()
labels.sort()
display_html(f"<b>Modalites de 'label'</b> - count: {len(labels)}")
display(list(labels))

<font class="comment">

**Notes**
* Background_without_leaves a ete intentionelment enleve
* les labels semblent avoir evolues par rapport a la liste cite ci-haut

In [ ]:
#
# distribution de label
#
FIGSIZE = (10, 4)

disease_count = pv_dataset["label"].value_counts()
disease_count.sort_values(inplace=True, ascending=False)

plt.figure(figsize=FIGSIZE)
disease_count.plot.bar(title="Effectifs de 'label'",
                       ylabel="count")
plt.show()

In [ ]:
#
# relation species/label
#
doi = pv_dataset[["label", "species"]]

disease_samples_per_species = doi.groupby("species").count()
disease_samples_per_species.sort_values("label", inplace=True, ascending=False)

In [ ]:
healthy_crit = doi["label"].str.contains("healthy")

healthy = doi[healthy_crit]
healthy = healthy \
            .groupby("species") \
            .count() \
            .rename(columns={"label": "healthy"})

not_healthy = doi[ ~healthy_crit ]
not_healthy = not_healthy \
                .groupby("species") \
                .count() \
                .rename(columns={"label": "autres"})

healthy_vs_other_per_species = pd.merge(healthy, 
                                        not_healthy,
                                        how="outer",
                                        on="species")

# for ease of comparison, keep same order 
# as disease_samples_per_species
healthy_vs_other_per_species = healthy_vs_other_per_species \
                                   .reindex(disease_samples_per_species \
                                   .index.to_list())

In [ ]:
fig = plt.figure(figsize=FIGSIZE)
fig.suptitle("Effectifs de 'label' par espece de plante")

# plt.subplot(121)
# disease_samples_per_species.plot.bar(ax=plt.gca(),
#                                      xlabel="",
#                                      ylabel="count",
#                                      legend=False)

plt.subplot(121)
healthy_vs_other_per_species.plot.bar(ax=plt.gca(),
                                      stacked=True, 
                                      xlabel="",
                                      ylabel="count")
plt.title("'healthy' vs autres", y=1.025)

plt.subplot(122)
_, _, pcts = plt.pie(x=disease_samples_per_species.values.flatten(),
                     labels=disease_samples_per_species.index.values,
                     autopct='%.0f%%',
                     radius=1.25,
                     wedgeprops={'linewidth': 1.0, 
                                 'edgecolor': 'white'})
plt.title("Toute maladies confondues", y=1.025)
plt.setp(pcts, color='white')

plt.tight_layout()
plt.show()

In [ ]:
def load_image(path):
    return pv_h5[path][...]

In [ ]:
#
# visualize some images
#
SAMPLEs_GRID = (2, 7)

# select some random samples
sample_indices = random.sample(range(pv_dataset.shape[0]), 
                               SAMPLEs_GRID[0] * SAMPLEs_GRID[1])
sample_indices.sort() 

samples = pv_dataset.loc[sample_indices].copy()

In [ ]:
samples["thumbnail"] = samples["image_path"].apply(load_image)

fig = plt.figure(figsize=FIGSIZE)
fig.suptitle("Echantillons maladies")

for i, (_, sample) in enumerate(samples.iterrows()):
    plt.subplot(SAMPLEs_GRID[0], SAMPLEs_GRID[1], i + 1)
    plt.axis("off")
    
    # adjust title: some disease have very long name
    species = sample['species']
    disease = sample['disease'].replace(species, "")
    disease = disease.split("_") 
    count = len(disease) 
    if count > 2:
        count //= 2
        disease0 = " ".join(disease[:count])
        disease1 = " ".join(disease[count:])
        disease = f"{disease0}\n{disease1}"
    else:
        disease = " ".join(disease)
    
    plt.title(f"{species}\n{disease}")    
    plt.imshow(sample["thumbnail"])
    plt.gca().title.set_size(10)

plt.tight_layout()
fig.subplots_adjust(hspace=0.5, top=0.8)
plt.show()

# Modelisation

Mettre un peu de bla bla pour justifier ce qu'on fait, citer les papiers et tout

**TODO**
* Train/test split
* Pipeline de traitement - Bag of Visual Words
    * Rescale en 256x256?
    * Remapper 0/1 et float
    * Threshold background/foreground    
    * Slicer les images
    * Detecter les features via SIFT/ORB
    * Passer dans un KMean
    * Mettre le tout dans notre dictionnaire

* Passer test data dans pipeline
    * Sparse vector pour une image non connue
    * Identifier avec K Nearest Neighboor

In [ ]:
#
# split train/test
#
train, test = train_test_split(pv_dataset, 
                               train_size=0.8, 
                               random_state=42, 
                               stratify=pv_dataset["label"])

In [ ]:
# validation split size
display_html(f"<b>Dataset:</b> <font style='color:red;'>{pv_dataset.size}</font>, "
             f"<b>Train:</b> <font style='color:red;'>{train.size}</font>, "
             f"<b>Test:</b> <font style='color:red;'>{test.size}</font> "
              "- 80% / 20% split")

In [ ]:
# validation split distribution
size = pv_dataset.shape[0]
frequencies = pd.DataFrame({"dataset": disease_count / size,
                            "train": train["label"].value_counts() / size,
                            "test": test["label"].value_counts() / size})
frequencies = frequencies.reindex(index=disease_count.index.to_list())

plt.figure(figsize=FIGSIZE)
frequencies.plot.bar(ax=plt.gca(),
                     stacked=True,
                     title="Distribution 'label' dataset vs train vs test\n"
                           "(couleurs meme taille dans chaque bar est mieux)",
                     ylabel="%")
plt.show()

In [ ]:
#
# construire les features
#
desc_factory = cv2.ORB_create()
feature_list = np.empty((0, 32), dtype=np.uint8)

progress = tqdm(iterable=pv_dataset.iterrows(),
                total=pv_dataset.shape[0],
                bar_format="{l_bar}{bar}{postfix}")

for index, row in progress:
    image_path = row["image_path"]
    progress.set_postfix_str(image_path)
    
    image = load_image(image_path)

    # tiling irait ici
    # detection backgroud irait ici
    _, descs = desc_factory.detectAndCompute(image, None)
    if descs is None:
        continue
    
    feature_list = np.append(feature_list, descs, axis=0)
    
    del image
    
#     if index > 100:
#         break

with open('descs.pkl', 'wb') as f:
    pickle.dump(list(feature_list), f)
    
# save the data to hdf5
print(pv_dataset.shape[0])
print(feature_list.shape)

# Evaluation

* Entrainement avec les stats